# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

**Содержание**

**[1. Обзор данных](#1)**

**[2. Деление исходных данных на обучающую, валидационную и тестовую выборки](#2)**

**[3. Исследование качества разных моделей](#3)**

**[4. Проверка качества модели на тестовой выборке](#4)**

**[5. Проверка модели на вменяемость](#5)**

**[Общие выводы](#6)**

**[Чек-лист готовности проекта](#7)**


**Заказчик.** Оператор мобильной связи «Мегалайн»

***Цель Заказчика.** Построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

**Цель исследования.** Построить модель для задачи классификации с максимально большим значением accuracy, которая выберет подходящий тариф. 

**Задачи:**

- Разделить исходные данные на обучающую, валидационную и тестовую выборки.

- Исследовать качество разных моделей, меняя гиперпараметры.

- Проверить качество модели на тестовой выборке.

- Проверить модели на вменяемость

**Входные данные от Заказчика.** Файл в формате .csv с указанием тарифного плана и ежемесячной статистикой голосовых вызовов, сообщений, интернет-трафика абонентов.

**Ожидаемый результат.** Построена модель с долей правильных ответов более 0.75

## Открытие и изучение файла. Обзор данных <a id='1'></a>

1. Открыть таблицы 
2. Изучить структуру данных по десяти первым строкам
3. Изучить структуру данных методом `info()`

**Описание данных**

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц

**Таблица**:

- `сalls` — количество звонков,
- `minutes` — суммарная длительность звонков в минутах,
- `messages` — количество sms-сообщений,
- `mb_used` — израсходованный интернет-трафик в Мб,
- `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [1]:
# Импотрт библиотек
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score

In [2]:
# Прочитаем файл
df = pd.read_csv('/datasets/users_behavior.csv')
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


### Вывод по обзору данных
В представленном файле 5 столбцов и 3214 строк. Все объекты имеют числовой тип. Пропусков данных нет. Предобработка данных не требуется.

Столбец is_ultra содержит целевой признак. Это будет использовано при последующем делении дата сета

## Деление исходных данных на обучающую, валидационную и тестовую выборки <a id='2'></a>

In [4]:
# Извлечем целевой признак в переменную target, а остальные признаки в features
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

In [5]:
# Отделим 40% данных для валидационной и тестовых выборок
features_train, features_valid_test, target_train, target_valid_test = (
    train_test_split(features, target, 
                     test_size=0.4, random_state=12345)
)

# Отделим 50% полученных данных от выборок features_valid_test и target_valid_test для разделения на валидационную и тестовую выборки
features_valid, features_test, target_valid, target_test = (
    train_test_split(features_valid_test, target_valid_test, 
                     test_size=0.5, random_state=12345)
)

In [6]:
print(f'features_train - {features_train.shape[0]} строк')
print(f'features_valid - {features_valid.shape[0]} строк')
print(f'features_test - {features_test.shape[0]} строк')

features_train - 1928 строк
features_valid - 643 строк
features_test - 643 строк


## Исследование качества разных моделей <a id='3'></a>

**Дерево решений**

Обучим модели на параметре `max_depth` от 1 до 10

Проверим методом `accuracy` на валидационной выборке

Лучшую модель сохраним для дальнейшего сравнения на тестовой выборке

In [7]:
best_model_tree = None
best_result_tree = 0
best_depth_tree = 0

for depth in range(1, 11):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_tree.fit(features_train, target_train)
    predication_tree = model_tree.predict(features_valid)
    result = accuracy_score(target_valid, predication_tree)
    print(f'При глубине дерева {depth} точность составляет {result:.4%}')
    if result > best_result_tree:
        best_model_tree = model_tree
        best_result_tree = result
        best_depth_tree = depth

print(f'accuracy наилучшей модели на валидационной выборке: {best_result_tree:.4%} глубина дерева: {best_depth_tree}')

При глубине дерева 1 точность составляет 75.4277%
При глубине дерева 2 точность составляет 78.2271%
При глубине дерева 3 точность составляет 78.5381%
При глубине дерева 4 точность составляет 77.9160%
При глубине дерева 5 точность составляет 77.9160%
При глубине дерева 6 точность составляет 78.3826%
При глубине дерева 7 точность составляет 78.2271%
При глубине дерева 8 точность составляет 77.9160%
При глубине дерева 9 точность составляет 78.2271%
При глубине дерева 10 точность составляет 77.4495%
accuracy наилучшей модели на валидационной выборке: 78.5381% глубина дерева: 3


**Случайный лес**

Обучим модели со следующими гиперпараметрами:
- с количеством деревьев: от 10 до 50 с шагом 5,
- с максимальной глубиной от 1 до 10

Проверим методом `accuracy` на валидационной выборке

Лучшую модель сохраним для дальнейшего сравнения на тестовой выборке

In [8]:
best_model_forest = None
best_result_forest = 0
best_est_forest = 0
best_depth_forest = 0

for est in range(10, 51, 5):
    for depth in range(1, 11):
        model_forest = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model_forest.fit(features_train, target_train)
        predication_forest = model_forest.predict(features_valid)
        result = accuracy_score(target_valid, predication_forest)
        if result > best_result_tree:
            best_model_forest = model_forest
            best_result_forest = result
            best_est_forest = est
            best_depth_forest = depth
            
print("accuracy наилучшей модели на валидационной выборке:", best_result_forest, "Количество деревьев:", best_est_forest, "Максимальная глубина:", best_depth_forest)

accuracy наилучшей модели на валидационной выборке: 0.7931570762052877 Количество деревьев: 50 Максимальная глубина: 10


**Логистическая регрессия**

Обучим модели на параметре `max_iter` от 100 до 1000 с шагом 100

Проверим методом `accuracy` на валидационной выборке

Лучшую модель сохраним для дальнейшего сравнения на тестовой выборке

In [9]:
best_model_reg = None
best_result_reg = 0
best_iter_reg = 0

for best_iter in range(100, 1001, 100):
    model_reg = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=best_iter)
    model_reg.fit(features_train, target_train)
    predication_reg = model_reg.predict(features_valid)
    result = accuracy_score(target_valid, predication_reg)
    print(f'При максимальном количестве итераций {best_iter} точность составляет {result:.4%}')
    if result > best_result_reg:
        best_model_reg = model_reg
        best_result_reg = result
        best_depth_reg = best_iter

print(f'accuracy наилучшей модели на валидационной выборке {best_result_reg:.4%}, Максимальное количество итераций обучения: {best_depth_reg}')

При максимальном количестве итераций 100 точность составляет 71.0731%
При максимальном количестве итераций 200 точность составляет 71.0731%
При максимальном количестве итераций 300 точность составляет 71.0731%
При максимальном количестве итераций 400 точность составляет 71.0731%
При максимальном количестве итераций 500 точность составляет 71.0731%
При максимальном количестве итераций 600 точность составляет 71.0731%
При максимальном количестве итераций 700 точность составляет 71.0731%
При максимальном количестве итераций 800 точность составляет 71.0731%
При максимальном количестве итераций 900 точность составляет 71.0731%
При максимальном количестве итераций 1000 точность составляет 71.0731%
accuracy наилучшей модели на валидационной выборке 71.0731%, Максимальное количество итераций обучения: 100


### Выводы по исследованию качества моделей

В рамках обучения моделей использовано три метода: `Дерево решений`, `Случайный лес` и `Логистическая регрессия`. Для выбора наиболее эффективного метода, в каждом из них использован различный набор гиперпараметров.

Для метода `Дерево решений` использован перебор гиперпараметра `max_depth` в диапазоне от 1 до 10. 

Для метода `Случайный лес` использован последовательный перебор двух гиперпараметров - `max_depth` в диапазоне от 1 до 10 и `n_estimators` в диапазоне от 10 до 50 с шагом 5.

Для метода `Логистическая регрессия` использован перебор гиперпараметра `max_iter` в диапазоне от 100 до 1000 с шагом 100.

Обучающая выборка составляет 60% генеральной совокупности. Все 120 обученных моделей были проверены на валидационной выборке методом `accuracy`. Валидационная выборка составляет 20% генеральной совокупности.

Наилучший показатель точности ответов 79.3% на валидационной выборке показала модель `Случайного леса` с гипрепараметрами максимальной глубины `max_depth` равной **10** и количеством деревьев `n_estimators` равным **50**.

## Проверка качества модели на тестовой выборке <a id='4'></a>

Проверим методом `accuracy` лучшую модель `best_model_forest` с полученными параметрами на тестовой выборке

In [10]:
# Проверим модель best_model_forest на тестовой выборке
predication_test = best_model_forest.predict(features_test)
result_test = accuracy_score(target_test, predication_test)
print(F'На тестовой выборке у лучшей модели {best_model_forest} \
accuracy составляет: {result_test:.4%}')

На тестовой выборке у лучшей модели RandomForestClassifier(max_depth=10, n_estimators=50, random_state=12345) accuracy составляет: 80.0933%


In [11]:
# Объединим обучающюю и валидационную выборки
target_train_all = pd.concat([target_train, target_valid], sort=False, axis=0)
features_train_all = features_train.append(features_valid, sort=False)
print(f'В выборке features_train_all {features_train_all.shape[0]} строк')

В выборке features_train_all 2571 строк


In [12]:
# Обучим модель еще раз н аобъедтненной выборке
best_model_forest.fit(features_train_all, target_train_all)
predication_forest_all = best_model_forest.predict(features_test)
result_test_all = accuracy_score(target_test, predication_forest_all)
print(F'На тестовой выборке у улучшеной модели {best_model_forest} \
accuracy составляет: {result_test_all:.4%}')

На тестовой выборке у улучшеной модели RandomForestClassifier(max_depth=10, n_estimators=50, random_state=12345) accuracy составляет: 80.0933%


### Вывод по проверке качества моделей на тестовой выборке

Полученные в предыдущем шаге модели протестированы методом `accuracy` на тестовой выборке. Тестовая выборка составляет 20% генеральной совокупности.

Протестировано три модели. Наилучший показатель точности ответов 80.1% подтвердила модель `Случайного леса` с гипрепараметрами максимальной глубины `max_depth` равной **10** и количеством деревьев `n_estimators` равным **50**.

## Проверка модели на адекватность (вменяемость) <a id='5'></a>

Методом `precision` и `recall` проверим отобранные модели на тестовой выборке

Сравним полученный результат с проверкой методом `accuracy`

In [13]:
# Создадим списки результатов проверки моделей методоми precision и recall
model_list = [best_model_tree, best_model_forest, best_model_reg]
result_accuracy_list = [best_result_tree, best_result_forest, best_result_reg]
result_precision_list = []
result_recall_list = []

# Проверим все модели на тестовой выборке методом precision
for models in model_list:
    predication_test = models.predict(features_test)
    result = average_precision_score(target_test, predication_test)
    result_precision_list.append(result)

# Проверим все модели на тестовой выборке методом recall
for models in model_list:
    predication_test = models.predict(features_test)
    result = recall_score(target_test, predication_test)
    result_recall_list.append(result)
    
#<img src="http://s3.amazonaws.com/pix.iemoji.com/images/emoji/apple/ios-12/256/waving-hand.png" align=left width=44, heigth=44>
#<div class="alert alert-info">
#<b> Заключительный комментарий</b>

#В целом мне все понравилось: твоя работа выполнена на хорошем уровне с минимальными помарками.
#Также работа выглядит аккуратной и хорошо оформленной.
#Вижу, что тебе дается python, и инструментарий Pandas.

#Ошибки, которые нужно обязательно исправить (красные комментарии):
    
#   -  Вывести название колонок в датафрейме до переименования
#   -  при подсчете количества прослушиваний по городам и дням недели нужно переписать группировку так, чтобы в качестве результата получать колонку, а не таблицу
#   -  
     
#Жду твоих исправлений :)
#</div>)


In [14]:
# Выведем полученные данные в таблицу
result_model = pd.DataFrame({
    'indicanor': ['tree', 'forest', 'regression'],
    'model': model_list, 
    'accuracy': result_accuracy_list,
    'precision': result_precision_list,
    'recall': result_recall_list
})

display(result_model)

,indicanor,model,accuracy,precision,recall
0,tree,"DecisionTreeClassifier(max_depth=3, random_sta...",0.785381,0.51192,0.458128
1,forest,"(DecisionTreeClassifier(max_depth=10, max_feat...",0.793157,0.55547,0.536946
2,regression,LogisticRegression(random_state=12345),0.710731,0.32297,0.039409


### Вывод по проверке модели на вменяемость

Для проверки моделей на вменяемость, отобранные модели проверены методами `precision` и `recall` на тестовой выборке. Тестовая выборка не менялась и составляет 20% генеральной совокупности.

Поскольку у нас всего два целевых варианта, то методы `recall` и `accuracy` показали одинаковые значения. Действительно, не важно какую часть тарифов `ultra` модель определила, как `ultra` (`recall`) или какую часть тарифов `ultra` модель определила, как верный ответ (`accuracy`).

Однако, `precision` показал довольно низкие значения – от 35.3% (у модели `Логистическая регрессия`) до 55,6% (у модели `Случайный лес`). Это говорит о высокой частоте ошибок, когда выбранные моделью тарифы `ultra`, действительно являются тарифом `ultra`.

Таким образом, созданные модели не склонны к совершению ошибок 2-го рода, но очень часто могут совершать ошибки 1-го рода, когда тариф на самом деле `ultra`, но модель его таким не считает.

На лучшей модели `Случайный лес` значение показателя `precision` 55,6% против значения `accuracy` 80,1%, очевидно, говорит о ее недостаточной точности. Однако, тот факт, что значение `precision` меньше значения `accuracy` подтверждает вменяемость построенной модели.

In [15]:
from sklearn.dummy import DummyClassifier

naive_model = DummyClassifier(strategy="most_frequent", random_state=12345)
naive_model.fit(features_train, target_train)
print(naive_model.score(features_train, target_train))

0.6924273858921162


## Общие выводы <a id='6'></a>

В рамках построения системы, способной проанализировать поведение клиентов и предложить пользователям новый тариф: `«Смарт»` или `«Ультра»`, обучены 120 моделей с использованием трех методов: `Дерево решений`, `Случайный лес` и `Логистическая регрессия`. 

Для задачи классификации с максимально большим значением `accuracy` при выборе наиболее эффективного метода, были использованы различные наборы гиперпараметров.

Обучающая выборка составила 60% генеральной совокупности. Все 120 обученных моделей были проверены на валидационной выборке методом `accuracy`, которая составила 20% генеральной совокупности.

Наилучший показатель точности ответов 79.3% на валидационной выборке показала модель `Случайного леса` с гипрепараметрами максимальной глубины `max_depth` равной **10** и количеством деревьев `n_estimators` равным **50**.

При тестировании трех лучших моделей на тестовой выборке, которая составила 20% генеральной совокупности, наилучший показатель точности ответов 80.1% подтвердила модель `Случайного леса`.

Для проверки моделей на вменяемость, отобранные модели проверены методами `precision` и `recall` на тестовой выборке. Тестовая выборка не менялась и составляет 20% генеральной совокупности.

`precision` показал довольно низкие значения – от 35.3% (у модели `Логистическая регрессия`) до 55,6% (у модели `Случайный лес`), что говорит о высокой частоте ошибок 1-го рода, когда тариф на самом деле `ultra`, но модель его таким не считает.

На лучшей модели `Случайный лес` значение показателя `precision` 55,6% против значения `accuracy` 80,1%, очевидно, говорит о ее недостаточной точности. Однако, тот факт, что значение `precision` меньше значения `accuracy` подтверждает вменяемость построенной модели.

Таким образом поставленная цель достигнута и построена модель с долей правильных ответов более 75%

## Чек-лист готовности проекта <a id='7'></a>

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
